In [1]:
from PIL import Image
import pytesseract
import configuration
import sqlite3

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
def ocr_processing(filename):
    
    text = pytesseract.image_to_string (Image.open(filename),config='--psm 12 --oem 3')  # We'll use Pillow's Image class to open the image and pytesseract to detect the string in the image
           
    return text



In [4]:
ocr_processing("./Reports/Mukesh.jpg")

'CITY PATHOLOGY LAB\n\nManchester M13 9NQ\n\nUK\n\nName\n\nMukesh Rao\n\nCollected\n\n:24/2/2020\n\nReceived\n\n: 24/2/2020\n\nLab No.\n\n134315810\n\nAge\n\nGender:\n\nReported\n\n:25/2/2020\n\nAc Status\n\nP\n\nRef By :\n\nReport Status\n\n: Final\n\nTest Name : FEVER PANEL\n\nCOMPLETE BLOOD COUNT (CBC)\n\n(Electrical Impedance & VCS,Photometry )\n\nResults\n\nUnits\n\nNormal Range\n\nHemoglobin\n\n13.30\n\ng/DL\n\n14-18\n\nPacked Cell Volume (PCV)\n\n42.80\n\nL/L\n\n35-90\n\nRBC Count\n\n5.27\n\nmil/mm3\n\n4.5-6\n\nMCV\n\n86.00\n\nfL\n\n82-98\n\nMCH\n\n28.20\n\npg\n\n27-31\n\nMCHC\n\n34\n\ng/dL\n\n32-36\n\nRed Cell DistributionWidth(RDW)\n\n13.50\n\n%\n\n11-16\n\nTotalLeukocyteCount (TLC)\n\n9.30\n\n1000/L\n\n4.5-11\n\nDifferential Leucocyte Count (DLC)\n\nSegmented Neutrophils\n\n66.40\n\n%\n\n40-75\n\nLymphocytes\n\n35.70\n\n%\n\n15-45\n\nMonocytes\n\n8.90\n\n%\n\n2-12\n\nEosinophils\n\n4.10\n\n%\n\n2-6\n\nBasophils\n\n0.40\n\n%\n\n0-1\n\nAbsolute Leucocyte Count\n\nNeutrophils\n\

In [5]:
def insert_into_db(patient_data_dict):
           
    con = sqlite3.connect('./Lab_Reports.db', timeout=10)
    if con:
        print("OK")
    else:
        print("NOT OK")
    
    cur = con.cursor()
    h=patient_data_dict['Hemoglobin']
    pcv=patient_data_dict['PCV']
    rbc=patient_data_dict['RBC']
    mcv=patient_data_dict['MCV']
    mch=patient_data_dict['MCH'] 
    mchc=patient_data_dict['MCHC']
    rdw=patient_data_dict['RDW']
    tlc=patient_data_dict['TLC']
    pc=patient_data_dict['Platelet_Count']
    sql_query = "CREATE TABLE CITY_PATHOLOGY_LAB_FEVER_PANEL (Patient_Name varchar(100) NOT NULL,Hemoglobin varchar(100) NOT NULL,PCV varchar(100) NOT NULL,RBC varchar(100) NOT NULL,MCV varchar(100) NOT NULL,MCH varchar(100) NOT NULL,MCHC varchar(100) NOT NULL,RDW varchar(100) NOT NULL,TLC varchar(100) NOT NULL,Platelet_Count varchar(100) NOT NULL,status,username varchar(100) NOT NULL,dates varchar(100) NOT NULL)"
    cur.execute(sql_query)
    if ((float(h)>=14 and float(h)<=18) and (float(pcv)>=35 and float(pcv)<=90) and (float(rbc)>=4.5 and float(rbc)<=6)and (float(mcv) >=82 and float(mcv)<=98) and (float(mch) >=27 and float(mch)<31) and (float(mchc) >=32 and float(mchc)<=36) and (float(rdw) >=11 and float(rdw)<=16) and (float(tlc) >=4.5 and float(tlc)<=11) and (float(pc) >=150 and float(pc)<=450)):
        sql_query = "INSERT INTO CITY_PATHOLOGY_LAB_FEVER_PANEL (Patient_Name,Hemoglobin,PCV,RBC,MCV,MCH,MCHC,RDW,TLC,Platelet_Count,status,username,dates) VALUES (?,?,?,?,?,?,?,?,?,?,'rejected',?,DateTime('now'))"
        cur.execute(sql_query,(patient_data_dict['Patient_Name'],patient_data_dict['Hemoglobin'],patient_data_dict['PCV'],  
        patient_data_dict['RBC'],patient_data_dict['MCV'],patient_data_dict['MCH'], 
        patient_data_dict['MCHC'],patient_data_dict['RDW'],patient_data_dict['TLC'], 
        patient_data_dict['Platelet_Count'], patient_data_dict['username']))

    elif((float(h)<14 or float(h)>18) or (float(pcv)<35 or float(pcv)>90) or (float(rbc)<4.5 or float(rbc)>6)or (float(mcv) <82 or float(mcv)>98) or (float(mch) <27 or float(mch)>31) or (float(mchc) <32 or float(mchc)>36) or (float(rdw) <11 or float(rdw)>16) or (float(tlc) <4.5 or float(tlc)>11) or (float(pc) <150 or float(pc)>450)):
        sql_query = "INSERT INTO CITY_PATHOLOGY_LAB_FEVER_PANEL (Patient_Name,Hemoglobin,PCV,RBC,MCV,MCH,MCHC,RDW,TLC,Platelet_Count,status,username,dates) VALUES (?,?,?,?,?,?,?,?,?,?,'accepted',?,DateTime('now'))"
        cur.execute(sql_query,(patient_data_dict['Patient_Name'],patient_data_dict['Hemoglobin'],patient_data_dict['PCV'],  
        patient_data_dict['RBC'],patient_data_dict['MCV'],patient_data_dict['MCH'], 
        patient_data_dict['MCHC'],patient_data_dict['RDW'],patient_data_dict['TLC'], 
        patient_data_dict['Platelet_Count'],patient_data_dict['username']))

    


    con.commit()
    con.close()  


In [8]:
def file_processing(filename): 
    
    try:      
        

        #Python-tesseract is a wrapper for Google’s Tesseract-OCR Engine. Here we connect with tesseract, which installed on system

        # pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

        #Passing image to ocr_processing function

        data = ocr_processing(filename)

        #Converting string to list by using split function, where \n is used as split parameter

        data_extracted_from_image = data.split('\n')

        #Creating new list list_without_null_values, where all null values are removed

        list_without_null_values = [i for i in data_extracted_from_image if i]

        #Data in list along with index

        #for index, value in enumerate(list_without_null_values): 
        #    print(index, value)

        #Check if the repory belongs to approved lab and test

        if (list_without_null_values[0] in configuration.approved_labs) and (list_without_null_values[20].replace("Test Name : ","") in configuration.approved_tests):

            #Creating dict with report details

            default_dict = {
            'Lab_Name':list_without_null_values[0],
            'Patient_Name':list_without_null_values[4],
            'Test_Name':list_without_null_values[20],
            'Hemoglobin':list_without_null_values[27],
            'PCV':list_without_null_values[31],
            'RBC':list_without_null_values[35],
            'MCV':list_without_null_values[39],
            'MCH':list_without_null_values[43],
            'MCHC':list_without_null_values[47],
            'RDW':list_without_null_values[51],
            'TLC':list_without_null_values[55],
            'Platelet_Count':list_without_null_values[101],
            'username':list_without_null_values[107]
            }

            # insert_into_db(default_dict)
            print(default_dict)
           
            text = "Submitted Reports of Patient Name :- " + default_dict['Patient_Name']
            
            return text
        
        else:

            text = "Invalid File Report"
            
            return text        
        
    except Exception as e:
        print(e)
        text = "Error occurred during file processing!!"
        
        return text     
       
    


In [11]:
file_processing("./Reports/Ramesh-Normal.jpg")

{'Lab_Name': 'CITY PATHOLOGY LAB', 'Patient_Name': 'Ramesh Patel', 'Test_Name': 'Test Name : FEVER PANEL', 'Hemoglobin': '15.30', 'PCV': '44.80', 'RBC': 'Buf', 'MCV': '85.00', 'MCH': '28.20', 'MCHC': '33.80', 'RDW': '13.50', 'TLC': '10.00', 'Platelet_Count': '300', 'username': 'ramesh@gmail.com'}


'Submitted Reports of Patient Name :- Ramesh Patel'